## ScenarioJudger

 - Reads a file from S3 containing json compliance scenarios of the format:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false,
      "non-compliant-reason": "The scenario violates..." 
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true,
      "non-compliant-reason": "" 
    }
  ]
}
```
 - Evaluates the veracity each scenario-detail based on RAGed NIST-based policies in Bedrock knowledgebase, comparing its determination against "is-compliant" in the json.
 - When its determination differs, generates json records:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false,
      "non-compliant-reason": "The scenario violates...",
      "judged-compliant": true,
      "judged-compliant-reason": "Considered the rules AC...  and scenario is not in violation..."
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":  
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true,
      "non-compliant-reason": "", 
      "judged-compliant": false,
      "judged-compliant-reason": "Scenario violates access control policy...",
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":   
    }
  ]
}
```
 - Stores json records back to S3


In [1]:
import boto3
import datetime
import json
import re
import time   # For rate limiting between API calls
from botocore.exceptions import ClientError
from compliance_utils import compliance_calculator, CALCULATOR_TOOL, COMPLIANCE_JUDGE_PROMPT
from pathlib import Path
from typing import List, Dict, Callable, Any

FOLDER_HOME: Path = Path('/home/sagemaker-user')
FOLDER_JUDGED_SCENARIOS: Path = FOLDER_HOME / 'data/judged_scenarios/'
BUCKET = '183023889407-us-east-1-compliance-rule-generator'
S3_SOURCE_SCENARIOS = 'scenarios/'  # Folder path in S3 where scenarios are stored
S3_SOURCE_POLICY_ALL = 'policies/markdown/all-policies-main/'
S3_JUDGED_SCENARIOS = 'scenarios-judged/'  # Folder path for results
AWS_REGION = 'us-east-1'
# KNOWLEDGE_BASE_ID = 'T8EW10IU3Z' - using s3 to retrieve policies, KB performance unsatisfactory
MAX_TOKENS = 4096
MAX_RETRIES_ON_THROTTLE = 5

# Tool configuration for Bedrock Converse API
# Forces the model to return structured JSON with specific schema, and use calculator tool
TOOL_CONFIG = {
    "tools": [
        {
            "toolSpec": {
                "name": "judged_scenario_json",
                "description": "Return judged compliance scenarios as JSON",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "scenarios": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "judged-compliant": {"type": "boolean"},
                                        "judged-compliant-reason": {"type": "string"}
                                    },
                                    "required": ["judged-compliant", "judged-compliant-reason"]
                                }
                            }
                        },
                        "required": ["scenarios"]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "compliance_calculator",
                "description": "Calculate and compare values with time, money, data, and percentage units",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "expression": {"type": "string", "description": "Expression like '800ms < 1s' or '4m > 3b'"}
                        },
                        "required": ["expression"]
                    }
                }
            }
        }
    ]
}
# CALCULATOR_TOOL["toolSpec"] references the calculator tool definition from compliance_calculator.py

# Initialize AWS Bedrock clients
# bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=AWS_REGION)  # For knowledge base retrieval
bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)  # For model inference

In [2]:
def bedrock_call_with_retry(func: Callable[[], Any], max_retries: int = MAX_RETRIES_ON_THROTTLE, base_delay: int = 2) -> Any:
    for attempt in range(max_retries):
        try:
            return func()
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                delay = base_delay * (2 ** attempt) # exponential backoff in the event of throttling
                print(f"Rate limit hit, waiting {delay}s...")
                time.sleep(delay)
            else:
                raise
    raise Exception("Max retries exceeded")

In [3]:
def load_scenarios_from_s3(input_bucket: str = BUCKET, s3_source_scenarios: str = S3_SOURCE_SCENARIOS, object_name: str = "scenarios.json") -> List[Dict]:
    """
    Load scenarios from S3 JSON file.
    """
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=input_bucket, Key=s3_source_scenarios+object_name)
    json_data = json.loads(response['Body'].read().decode('utf-8'))
    return json_data["scenarios"]


In [4]:
def save_scenarios_to_s3(scenarios: List[Dict], output_bucket: str = BUCKET, s3_judged_scenarios: str = S3_JUDGED_SCENARIOS, object_name: str = "scenarios.json"):
    """
    Save generated scenarios to a S3.
    """
    s3 = boto3.client('s3')
    json_data = json.dumps({"scenarios": scenarios}, indent=2)
    s3.put_object(Bucket=output_bucket, Key=s3_judged_scenarios+object_name, Body=json_data)

In [5]:
def retrieve_policies_by_id(bucket:str, folder:str, policy_ids: List[str]) -> str:
    """
    Retrieve specific policy documents from s3.
    """

    s3 = boto3.client('s3')
    policies = []
    for policy_id in policy_ids:
        response = s3.get_object(Bucket=bucket, Key=folder + policy_id + ".md")
        content = response['Body'].read().decode('utf-8')
        policies.append(f"{policy_id}:\n{content}")
    
    return "\n\n".join(policies)
    

In [6]:
def get_policies_for_scenario(scenario: Dict) -> str:
    """Extract policy IDs from scenario and retrieve policy content from S3"""
    
    # Extract policy IDs from scenario text
    policy_match = re.search(r'Policies referenced: (.+)', scenario["scenario-detail"])
    if not policy_match:
        print(f"No policies referenced in scenario: {scenario['scenario-detail']}")
        return ""
    
    # Parse policy IDs
    policy_ids = [p.strip() for p in policy_match.group(1).split(',')]
    
    # Retrieve policy documents from S3
    s3 = boto3.client('s3')
    policies = []
    for policy_id in policy_ids:
        response = s3.get_object(Bucket=BUCKET, Key=S3_SOURCE_POLICY_ALL + policy_id + ".md")
        content = response['Body'].read().decode('utf-8')
        policies.append(f"{policy_id}:\n{content}")
    
    return "\n\n".join(policies)
    

In [7]:
def judge_with_claude(
    prompt: str,
    model_id: str,
    temperature: float
) -> Dict:
    
    messages=[{"role": "user", "content": [{"text": prompt}]}]
    input_tokens = 0
    output_tokens = 0
    result = {}
    
    while True:
        response = bedrock_call_with_retry(
            lambda: bedrock_runtime.converse(modelId=model_id, messages=messages,toolConfig=TOOL_CONFIG,
                inferenceConfig={
                    "maxTokens": MAX_TOKENS,
                    "temperature": temperature 
                }
            )
        )

        # track per-scenario token usage
        usage = response.get('usage', {})
        input_tokens += usage.get('inputTokens', 0)
        output_tokens += usage.get('outputTokens', 0)
        
        if response['stopReason'] == 'tool_use':
            tool_results = []
            for content_block in response['output']['message']['content']:
                if 'toolUse' in content_block:
                    tool_name = content_block['toolUse']['name']
                    tool_use_id = content_block['toolUse']['toolUseId']
                    
                    if tool_name == 'compliance_calculator':
                        expression = content_block['toolUse']['input']['expression']                         
                        calc_result = compliance_calculator(expression)
                        print("=" * 60)
                        print(f"Compliance calculator expression: {expression}")
                        print(f"Compliance calculator result: {calc_result}")
                        print("=" * 60)
                        tool_results.append({
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [{"text": calc_result}]
                            }
                        })
                    elif tool_name == 'judged_scenario_json':
                        tool_result = content_block['toolUse']['input']
                        result["judged-compliant"] = tool_result['scenarios'][0]['judged-compliant']
                        result["judged-compliant-reason"] = tool_result['scenarios'][0]['judged-compliant-reason']
                        break
            
            if tool_results:
                messages.append({"role": "assistant", "content": response['output']['message']['content']})
                messages.append({"role": "user", "content": tool_results})
            else:
                break
        else:
            break

    result.update({
        "llm-judge-input-tokens" : input_tokens,
        "llm-judge-output-tokens": output_tokens,
        "llm-judge-total-tokens" : input_tokens + output_tokens
    })
        
    return result
    


In [8]:
def judge_with_nova(prompt: str, model_id: str, temperature: float) -> Dict:
    enhanced_prompt = f"""{prompt}

Respond with a JSON object in this exact format:
{{
  "judged-compliant": true/false,
  "judged-compliant-reason": "detailed explanation"
}}"""
    
    body = {
        "messages": [{"role": "user", "content": [{"text": enhanced_prompt}]}],
        "inferenceConfig": {
            "maxTokens": MAX_TOKENS,
            "temperature": temperature
        }
    }
    
    response = bedrock_call_with_retry(
        lambda: bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(body),
            contentType="application/json"
        )
    )
    
    response_body = json.loads(response['body'].read())
    content = response_body['output']['message']['content'][0]['text']
    
    # Extract JSON from response
    try:
        # Find JSON in response text
        start = content.find('{')
        end = content.rfind('}') + 1
        json_str = content[start:end]
        result = json.loads(json_str)
        
        return {
            "judged-compliant": result["judged-compliant"],
            "judged-compliant-reason": result["judged-compliant-reason"],
            "llm-judge-input-tokens": response_body.get('usage', {}).get('inputTokens', 0),
            "llm-judge-output-tokens": response_body.get('usage', {}).get('outputTokens', 0),
            "llm-judge-total-tokens": response_body.get('usage', {}).get('inputTokens', 0) + response_body.get('usage', {}).get('outputTokens', 0)
        }
    except (json.JSONDecodeError, KeyError) as e:
        return {
            "judged-compliant": False,
            "judged-compliant-reason": f"Error parsing Nova response: {str(e)}",
            "llm-judge-input-tokens": 0,
            "llm-judge-output-tokens": 0,
            "llm-judge-total-tokens": 0
        }


In [9]:
def judge_scenarios(
    source_scenarios: List[Dict],
    model_arn: str, 
    temperature: float = None,
    max_scenarios: int = None
) -> List[Dict]:
    
    # Extract model ID and detect model type
    model_id = model_arn.split('/')[-1] if '/' in model_arn else model_arn
    is_nova_model = "nova" in model_id.lower()
    
    judged_scenarios = []
    for scenario in source_scenarios[:max_scenarios] if max_scenarios else source_scenarios:
        
        # Get policy context from S3 for this scenario
        retrieved_policies = get_policies_for_scenario(scenario)
        if not retrieved_policies:
            continue  # Skip scenarios without policy references
        
        prompt = COMPLIANCE_JUDGE_PROMPT.format(
            retrieved_policies=retrieved_policies,
            scenario_detail=scenario["scenario-detail"]
        )
        
        if is_nova_model:
            # Nova: Use invoke_model with JSON response
            judged_scenario_detail = judge_with_nova(prompt, model_id, temperature)
        else:
            # Claude: Use converse with tools
            judged_scenario_detail = judge_with_claude(prompt, model_id, temperature)
        
        # Add metadata
        judged_scenario = scenario.copy()
        judged_scenario.update(judged_scenario_detail)
        judged_scenario.update({
            "judged-dtm": datetime.datetime.now().isoformat(),
            "llm-judge": model_id,
            "llm-judge-temp": temperature
        })
        
        judged_scenarios.append(judged_scenario)
    
    return judged_scenarios


In [10]:
def judge_scenarios_old(
    source_scenarios: List[Dict],
    model_arn: str, 
    kb_id: str,
    temperature : float = None,
    max_scenarios: int = None
) -> List[Dict]:
    """
    Process scenarios and add judgment fields.
    """

    # Limit scenarios if max_scenarios is specified
    scenarios_to_process = source_scenarios[:max_scenarios] if max_scenarios else source_scenarios
    
    # Extract model ID from ARN (Converse API requires model ID, not full ARN)
    model_id = model_arn.split('/')[-1] if '/' in model_arn else model_arn
    
    judged_scenarios = []
    for scenario in scenarios_to_process:
        judged_scenario = scenario.copy()

        # Extract policy IDs from scenario and pull the policy data from the knowledge base
        import re
        policy_match = re.search(r'Policies referenced: (.+)', scenario["scenario-detail"])
        if policy_match:
            policy_ids = [p.strip() for p in policy_match.group(1).split(',')]
            retrieved_policies = retrieve_policies_by_id(BUCKET, S3_SOURCE_POLICY_ALL, policy_ids)
        else:
            print(f"No policies referenced in scenario: {scenario['scenario-detail']}")
            continue
                
        prompt = COMPLIANCE_JUDGE_PROMPT.format(
            retrieved_policies=retrieved_policies,
            scenario_detail=scenario["scenario-detail"]
        )

        messages=[{"role": "user", "content": [{"text": prompt}]}]
        input_tokens = 0
        output_tokens = 0
        
        while True:
            response = bedrock_call_with_retry(
                lambda: bedrock_runtime.converse(
                    modelId=model_id,
                    messages=messages,
                    toolConfig=TOOL_CONFIG,
                    inferenceConfig={
                        "maxTokens": MAX_TOKENS,
                        "temperature": temperature 
                    }
                )
            )

            # track per-scenario token usage
            usage = response.get('usage', {})
            input_tokens += usage.get('inputTokens', 0)
            output_tokens += usage.get('outputTokens', 0)
            
            if response['stopReason'] == 'tool_use':
                tool_results = []
                for content_block in response['output']['message']['content']:
                    if 'toolUse' in content_block:
                        tool_name = content_block['toolUse']['name']
                        tool_use_id = content_block['toolUse']['toolUseId']
                        
                        if tool_name == 'compliance_calculator':
                            expression = content_block['toolUse']['input']['expression']                         
                            result = compliance_calculator(expression)
                            print("=" * 60)
                            print(f"Compliance calculator expression: {expression}" )
                            print(f"Compliance calculator result: {result}" )
                            print("=" * 60)
                            tool_results.append({
                                "toolResult": {
                                    "toolUseId": tool_use_id,
                                    "content": [{"text": result}]
                                }
                            })
                        elif tool_name == 'judged_scenario_json':
                            tool_result = content_block['toolUse']['input']
                            judged_scenario["judged-compliant"] = tool_result['scenarios'][0]['judged-compliant']
                            judged_scenario["judged-compliant-reason"] = tool_result['scenarios'][0]['judged-compliant-reason']
                            break
                
                if tool_results:
                    messages.append({"role": "assistant", "content": response['output']['message']['content']})
                    messages.append({"role": "user", "content": tool_results})
                else:
                    break
            else:
                break

        judged_scenario["judged-dtm"] = datetime.datetime.now().isoformat()
        judged_scenario["llm-judge"] = model_arn.split('/')[-1]
        judged_scenario["llm-judge-temp"] = temperature 
        judged_scenario["llm-judge-input-tokens"] = input_tokens
        judged_scenario["llm-judge-output-tokens"] = output_tokens
        judged_scenario["llm-judge-total-tokens"] = input_tokens + output_tokens
        judged_scenarios.append(judged_scenario)
    
    return judged_scenarios

In [11]:
def save_scenarios_to_file(scenarios: List[Dict], output_path: Path):
    
    # Print scenarios to console for immediate review
    print(json.dumps(scenarios, indent=2))

    # Create parent directories if they don't exist
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Save to file with metadata and statistics
    with open(output_path, 'w') as f:
        json.dump({
            'total_scenarios': len(scenarios),
            'compliant_count': sum(1 for s in scenarios if s['is-compliant']),
            'non_compliant_count': sum(1 for s in scenarios if not s['is-compliant']),
            'judged_compliant_count': sum(1 for s in scenarios if s['judged-compliant']),
            'judged_non_compliant_count': sum(1 for s in scenarios if not s['judged-compliant']),
            'scenarios': scenarios
        }, f, indent=2)

In [18]:
def main():
    
    judger_models = [
        {
            'name': 'claude_3_7_sonnet',
            'arn': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0',
            'temperature': 0.1
        },
        {
            'name': 'claude_4_sonnet',
            'arn': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-sonnet-4-20250514-v1:0',
            'temperature': 0.0
        },
        {
            'name': 'claude_opus_4_5',
            'arn': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-opus-4-5-20251101-v1:0',
            'temperature': 0.0
        },
        {
            'name': 'nova_premier',
            'arn': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-premier-v1:0',
            'temperature': 0.1
        },
        {
            'name': 'nova_2_lite',
            'arn': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-2-lite-v1:0',
            'temperature': 0.2
        }
    ]
    
    source_scenarios_file = "scenarios.json"  
    source_scenarios = load_scenarios_from_s3(BUCKET, S3_SOURCE_SCENARIOS, source_scenarios_file)

    for model in judger_models:
        print(f"Processing with model: {model['name']} (temp: {model['temperature']})")
    
        judged_scenarios = judge_scenarios(
            source_scenarios=source_scenarios,
            model_arn=model['arn'],
            temperature =model['temperature'],
            max_scenarios=8
        )
        
        judged_scenarios_file = f"judged_scenarios_{model['name']}_temp{model['temperature']}.json"
        
        save_scenarios_to_file(judged_scenarios, FOLDER_JUDGED_SCENARIOS / judged_scenarios_file)
        save_scenarios_to_s3(judged_scenarios, BUCKET, S3_JUDGED_SCENARIOS, judged_scenarios_file)

        print(f"Completed {model['name']}: {len(judged_scenarios)} scenarios processed")


In [19]:
main()

Processing with model: claude_3_7_sonnet (temp: 0.1)
Compliance calculator expression: 80 hours > 72 hours
Compliance calculator result: True
Compliance calculator expression: 60 days < 90 days
Compliance calculator result: True
Compliance calculator expression: 256 bit > 128 bit
Compliance calculator result: True
Compliance calculator expression: 3 days < 5 days
Compliance calculator result: True
Compliance calculator expression: 96 hours > 72 hours
Compliance calculator result: True
Compliance calculator expression: 192 bits >= 128 bits
Compliance calculator result: True
Compliance calculator expression: 2 business days <= 5 business days
Compliance calculator result: True
Compliance calculator expression: 75 days < 90 days
Compliance calculator result: True
Compliance calculator expression: 120 hours > 72 hours
Compliance calculator result: True
Compliance calculator expression: 384 bits > 128 bits
Compliance calculator result: True
Compliance calculator expression: 4 days < 5 days


In [17]:
bedrock_client = boto3.client('bedrock', region_name=AWS_REGION)
response = bedrock_client.list_inference_profiles()
for profile in response['inferenceProfileSummaries']:
    if 'nova' in profile['inferenceProfileName'].lower():
        print(f"ARN: {profile['inferenceProfileArn']}")


ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-premier-v1:0
ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-pro-v1:0
ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-lite-v1:0
ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-micro-v1:0
ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.amazon.nova-2-lite-v1:0
ARN: arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-2-lite-v1:0
